Data prep purchase orders


*** units: NaN, PUND and KG -> transform all to KG

*** what to do with negative values

*** date_time columns have various time zones (+00:00, +01:00, +02:00)

*** delivery_date = expected delivery date, does this correspond to date_arrival in receivals.csv?

*** quantity er ikke heltallig (1414)?

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
purchase = pd.read_csv("/home/celinaha/ml_group21/data/kernel/purchase_orders.csv").copy()

In [44]:
purchase.head()

,purchase_order_id,purchase_order_item_no,quantity,delivery_date,product_id,product_version,created_date_time,modified_date_time,unit_id,unit,status_id,status
0,1,1,-14.0,2003-05-12 00:00:00.0000000 +02:00,91900143,1,2003-05-12 10:00:48.0000000 +00:00,2004-06-15 06:16:18.0000000 +00:00,NaN,NaN,2,Closed
1,22,1,23880.0,2003-05-27 00:00:00.0000000 +02:00,91900160,1,2003-05-27 12:42:07.0000000 +00:00,2012-06-29 09:41:13.0000000 +00:00,NaN,NaN,2,Closed
2,41,1,0.0,2004-03-08 00:00:00.0000000 +01:00,91900143,1,2004-03-08 13:44:31.0000000 +00:00,2012-07-04 13:51:02.0000000 +00:00,NaN,NaN,2,Closed
3,61,1,0.0,2004-03-10 00:00:00.0000000 +01:00,91900143,1,2004-03-10 11:39:06.0000000 +00:00,2012-07-04 13:50:59.0000000 +00:00,NaN,NaN,2,Closed
4,141,10,25000.0,2004-10-28 00:00:00.0000000 +02:00,91900143,1,2004-10-22 12:21:54.0000000 +00:00,2012-07-04 13:50:55.0000000 +00:00,NaN,NaN,2,Closed


In [45]:
purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33171 entries, 0 to 33170
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   purchase_order_id       33171 non-null  int64  
 1   purchase_order_item_no  33171 non-null  int64  
 2   quantity                33171 non-null  float64
 3   delivery_date           33171 non-null  object 
 4   product_id              33171 non-null  int64  
 5   product_version         33171 non-null  int64  
 6   created_date_time       33171 non-null  object 
 7   modified_date_time      32679 non-null  object 
 8   unit_id                 33127 non-null  float64
 9   unit                    33127 non-null  object 
 10  status_id               33171 non-null  int64  
 11  status                  33171 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 3.0+ MB


In [46]:
purchase.nunique()

purchase_order_id          8135
purchase_order_item_no       94
quantity                   4997
delivery_date              2332
product_id                   77
product_version               9
created_date_time         14657
modified_date_time        22845
unit_id                       2
unit                          2
status_id                     3
status                        3
dtype: int64

In [47]:
purchase['unit'].unique()

array([nan, 'KG', 'PUND'], dtype=object)

In [48]:
purchase['status'].unique()

array(['Closed', 'Deleted', 'Open'], dtype=object)

In [49]:
purchase[purchase['status']=='Deleted']

,purchase_order_id,purchase_order_item_no,quantity,delivery_date,product_id,product_version,created_date_time,modified_date_time,unit_id,unit,status_id,status
34,467,1,100.0,2020-10-01 00:00:00.0000000 +02:00,91901460,1,2020-10-01 10:51:22.0000000 +00:00,2020-10-01 12:27:55.0000000 +00:00,NaN,NaN,3,Deleted
42,528,1,25000.0,2023-09-13 00:00:00.0000000 +02:00,91901471,2,2023-09-13 14:37:46.0000000 +00:00,2025-03-04 14:05:52.0000000 +00:00,NaN,NaN,3,Deleted
43,547,1,25000.0,2023-10-04 00:00:00.0000000 +02:00,91901471,2,2023-10-04 09:03:42.0000000 +00:00,2023-10-04 14:58:43.0000000 +00:00,NaN,NaN,3,Deleted
487,212998,190,13480.0,2005-02-28 00:00:00.0000000 +01:00,91900170,1,2005-03-03 13:34:49.0000000 +00:00,2005-10-19 09:32:06.0000000 +00:00,40.0,KG,3,Deleted
569,213207,60,75000.0,2005-03-31 00:00:00.0000000 +02:00,91900296,1,2005-01-26 11:30:31.0000000 +00:00,2005-03-15 09:34:54.0000000 +00:00,40.0,KG,3,Deleted
...,...,...,...,...,...,...,...,...,...,...,...,...
32668,328100,10,36000.0,2024-08-31 00:00:00.0000000 +02:00,91901370,3,2024-07-10 10:42:01.0000000 +00:00,2024-09-23 10:40:07.0000000 +00:00,40.0,KG,3,Deleted
32701,328341,10,25000.0,2024-07-26 00:00:00.0000000 +02:00,91901320,5,2024-07-29 12:06:13.0000000 +00:00,2024-10-01 10:35:33.0000000 +00:00,40.0,KG,3,Deleted
32841,329183,10,25000.0,2024-09-30 00:00:00.0000000 +02:00,91900274,7,2024-09-13 11:57:50.0000000 +00:00,2024-09-16 10:27:25.0000000 +00:00,40.0,KG,3,Deleted
32984,330446,10,100000.0,2025-03-31 00:00:00.0000000 +02:00,91900143,9,2024-11-21 10:40:50.0000000 +00:00,2024-12-19 11:22:55.0000000 +00:00,40.0,KG,3,Deleted


Removing duplicates:

In [50]:
print(purchase.duplicated().any())

False


Fixing DateTime, and Date:

In [62]:
purchase['created_date_time'] = pd.to_datetime(purchase['created_date_time'], errors='coerce')
purchase['modified_date_time'] = pd.to_datetime(purchase['modified_date_time'], errors='coerce')

In [63]:
purchase['delivery_date'] = pd.to_datetime(purchase['delivery_date'], errors='coerce')


In [64]:
purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33171 entries, 0 to 33170
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype                    
---  ------                  --------------  -----                    
 0   purchase_order_id       33171 non-null  int64                    
 1   purchase_order_item_no  33171 non-null  int64                    
 2   quantity                33171 non-null  float64                  
 3   delivery_date           19132 non-null  datetime64[ns, UTC+02:00]
 4   product_id              33171 non-null  int64                    
 5   product_version         33171 non-null  int64                    
 6   created_date_time       33171 non-null  datetime64[ns, UTC]      
 7   modified_date_time      32679 non-null  datetime64[ns, UTC]      
 8   unit_id                 33127 non-null  float64                  
 9   unit                    33127 non-null  object                   
 10  status_id               33171 non-

Fixing different timezones:

Removing negative values:

In [54]:
(purchase.select_dtypes(include=['number']) < 0).any()

purchase_order_id         False
purchase_order_item_no    False
quantity                   True
product_id                False
product_version           False
unit_id                   False
status_id                 False
dtype: bool

In [55]:
purchase[purchase['quantity']<0]

,purchase_order_id,purchase_order_item_no,quantity,delivery_date,product_id,product_version,created_date_time,modified_date_time,unit_id,unit,status_id,status
0,1,1,-14.0,2003-05-12 00:00:00+02:00,91900143,1,2003-05-12 10:00:48+00:00,2004-06-15 06:16:18+00:00,NaN,NaN,2,Closed
59,205323,10,-380.0,2003-01-31 00:00:00+01:00,1024,1,2003-01-28 13:30:23+00:00,2004-06-15 06:08:40+00:00,40.0,KG,2,Closed
60,205443,10,-390.0,2003-02-10 00:00:00+01:00,1024,1,2003-02-10 09:31:21+00:00,2004-06-15 06:14:46+00:00,40.0,KG,2,Closed
61,206151,10,-260.0,2003-04-21 00:00:00+02:00,1024,1,2003-04-14 09:31:14+00:00,2004-06-15 06:14:50+00:00,40.0,KG,2,Closed
66,206386,10,-340.0,2003-05-14 00:00:00+02:00,1024,1,2003-05-09 08:30:53+00:00,2004-06-15 06:14:54+00:00,40.0,KG,2,Closed
208,209842,10,-8580.0,2004-04-30 00:00:00+02:00,91900170,1,2004-04-28 08:32:31+00:00,2004-06-15 06:16:36+00:00,40.0,KG,2,Closed


Missing values; handling NaN in unit

In [56]:
purchase.isna().sum()

purchase_order_id           0
purchase_order_item_no      0
quantity                    0
delivery_date               0
product_id                  0
product_version             0
created_date_time           0
modified_date_time        492
unit_id                    44
unit                       44
status_id                   0
status                      0
dtype: int64

In [57]:
purchase[purchase['unit']=='PUND']

,purchase_order_id,purchase_order_item_no,quantity,delivery_date,product_id,product_version,created_date_time,modified_date_time,unit_id,unit,status_id,status
20597,274174,10,1079.998444,2015-06-25 00:00:00+02:00,1414,2,2015-06-16 13:10:15+00:00,NaT,43.0,PUND,1,Open
21398,277831,10,1079.998444,2016-01-19 00:00:00+01:00,1414,2,2016-01-14 10:51:26+00:00,NaT,43.0,PUND,1,Open
21583,278932,10,1079.998444,2016-03-08 00:00:00+01:00,1414,2,2016-03-01 15:15:10+00:00,NaT,43.0,PUND,1,Open
21906,279515,10,1079.998444,2016-04-14 00:00:00+02:00,1414,2,2016-04-07 10:29:09+00:00,NaT,43.0,PUND,1,Open
22121,280240,10,2159.996888,2016-05-17 00:00:00+02:00,1414,2,2016-05-11 15:27:56+00:00,NaT,43.0,PUND,1,Open
22666,282355,10,2159.996888,2016-09-20 00:00:00+02:00,1414,2,2016-09-13 15:23:26+00:00,NaT,43.0,PUND,1,Open
23817,286407,10,1079.998444,2017-04-28 00:00:00+02:00,1414,2,2017-04-25 14:20:17+00:00,NaT,43.0,PUND,1,Open


*** Product id 1414 has units pund

After fixing datetime:

In [58]:
purchase.isna().sum()

purchase_order_id           0
purchase_order_item_no      0
quantity                    0
delivery_date               0
product_id                  0
product_version             0
created_date_time           0
modified_date_time        492
unit_id                    44
unit                       44
status_id                   0
status                      0
dtype: int64

In [61]:
purchase[purchase['delivery_date'].isna()]

,purchase_order_id,purchase_order_item_no,quantity,delivery_date,product_id,product_version,created_date_time,modified_date_time,unit_id,unit,status_id,status


In [60]:
purchase[purchase['unit'].isna()]

,purchase_order_id,purchase_order_item_no,quantity,delivery_date,product_id,product_version,created_date_time,modified_date_time,unit_id,unit,status_id,status
0,1,1,-14.0,2003-05-12 00:00:00+02:00,91900143,1,2003-05-12 10:00:48+00:00,2004-06-15 06:16:18+00:00,NaN,NaN,2,Closed
1,22,1,23880.0,2003-05-27 00:00:00+02:00,91900160,1,2003-05-27 12:42:07+00:00,2012-06-29 09:41:13+00:00,NaN,NaN,2,Closed
2,41,1,0.0,2004-03-08 00:00:00+01:00,91900143,1,2004-03-08 13:44:31+00:00,2012-07-04 13:51:02+00:00,NaN,NaN,2,Closed
3,61,1,0.0,2004-03-10 00:00:00+01:00,91900143,1,2004-03-10 11:39:06+00:00,2012-07-04 13:50:59+00:00,NaN,NaN,2,Closed
4,141,10,25000.0,2004-10-28 00:00:00+02:00,91900143,1,2004-10-22 12:21:54+00:00,2012-07-04 13:50:55+00:00,NaN,NaN,2,Closed
5,161,10,6000.0,2005-03-11 00:00:00+01:00,91900143,1,2005-03-11 13:53:25+00:00,2012-07-04 13:50:49+00:00,NaN,NaN,2,Closed
6,161,20,15000.0,2006-03-27 00:00:00+02:00,91900143,1,2006-03-27 11:04:44+00:00,2012-07-04 13:50:52+00:00,NaN,NaN,2,Closed
7,361,10,150000.0,2012-07-31 00:00:00+02:00,91900296,1,2012-07-04 13:53:29+00:00,2014-07-29 10:58:02+00:00,NaN,NaN,2,Closed
8,361,20,150000.0,2012-07-31 00:00:00+02:00,91900170,1,2012-07-04 13:55:14+00:00,2014-07-29 10:58:03+00:00,NaN,NaN,2,Closed
9,361,30,150000.0,2012-07-31 00:00:00+02:00,91901050,1,2012-07-09 07:52:43+00:00,2014-07-29 10:58:05+00:00,NaN,NaN,2,Closed
